<center>
<h1 style="font-family:verdana">
 🏘 Sistema de diàleg basat en regles 🏘 <h1>








<p>   🎯 <b>Objectiu</b>: en aquesta pràctica aprendrem a crear un assistent senzill de compra d'habitatges. El sistema haurà d'identificar i mostrar les cases que coincideixen amb les preferències de l'usuari. A poc a poc anirem afegint-hi funcionalitats, perquè el sistema siga més complet. </p>

<p> ✨ <b>Contingut</b>: en primer lloc, començarem amb un exemple senzill en què el sistema llançarà unes preguntes i respostes predefinides i l'usuari haurà d'escollir. A poc a poc anirem afegint-hi funcionalitats, perquè el sistema siga més complet. </p>

✏ <b>Exercicis</b>: en cada secció anireu trobant exercicis que haureu d'anar resolent.

---

<h2> Índex </h2>


1. [Fitxer JSON](#section-one)
  * [Exercici 1](#ex-one)
2. [Sistema de diàleg senzill](#section-two)
  * [Exercici 2](#ex-two)
  * [Exercici 3](#ex-three)
  * [Exercici 4](#ex-four)
3. [Millorem el sistema de diàleg](#section-three)
  * [Exercici 5](#ex-five)
  * [Exercici 6](#ex-six)
4. [Lliurable](#section-four)
---


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from nltk.tokenize.treebank import TreebankWordTokenizer
import nltk
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer

import json
import sys

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


<h1><a name="section-one"> 1. Fitxer JSON </a></h1>

En primer lloc, carregarem el fitxer JSON per analitzar-ne l'estructura.

In [ ]:
with open('/content/drive/MyDrive/3r_1/TVD/P1/dades/house_data.json') as f:
  data = json.load(f)

print('Start message: ', data['start_message'])
print('End message: ', data['end_message'])
print('Questions: ', data['questions'])
print('House: ', data['houses'])

Start message:  Welcome to the House Buying Assistant!
End message:  Thank you for using the House Buying Assistant. Goodbye!
Questions:  [{'question': 'How many bedrooms do you need?', 'type': 'numerical', 'prompt': 'Enter your choice (1 - 5 bedrooms): ', 'answer_key': 'bedrooms'}, {'question': 'How many bathrooms do you need?', 'type': 'numerical', 'prompt': 'Enter your choice (1 - 3 bathrooms): ', 'answer_key': 'bathrooms'}, {'question': 'What is your budget for the house?', 'type': 'numerical', 'prompt': 'Enter your choice (1k - 999k euros):', 'answer_key': 'price'}, {'question': 'How many square meters do you need?', 'type': 'numerical', 'prompt': 'Enter your choice (30 - 200 meters)', 'answer_key': 'square_meters'}, {'question': 'Which city or neighborhood would you prefer?', 'type': 'multichoice', 'prompt': 'Enter your choice', 'answer_key': 'location'}]
House:  [{'id': 1, 'type': 'rent', 'bedrooms': '3', 'bathrooms': '2', 'price': '1200', 'square_meters': '100', 'floor': '3', '

Podeu veure com al fitxer JSON podem trobar el missatge de benvinguda i el missatge de comiat. Les diferents preguntes predefinides que el sistema demanarà a l'usuari. I les cases que l'agència té disponibles per a oferir a l'usuari.

Si no acabeu d'entendre el format, podeu copiar el contingut del fitxer JSON [aquí](https://jsonviewer.stack.hu/) per visualitzar millor l'estructura.

---



 <h1><a name="ex-one"><center> ✏ Exercici 1 ✏</a></h1>


En aquest primer exercici us demanem que afegiu una nova casa manualment al fitxer JSON amb l'id 26. La resta de camps els podeu emplenar com vulgueu.

Si ho heu fet correctament podreu veure al següent *print* les dades que heu introduït.

In [ ]:
new = {
    "id": 26,
	  "type":"sale",
    "bedrooms": "1",
    "bathrooms": "1",
    "price": "100k",
    "square_meters": "50",
	  "floor":"1",
	  "elevator":"No",
	  "commercial_use":"No",
	  "terrace": "No",
    "location": "Santa Coloma de Gramenet"
}
data['houses'].append(new)


In [ ]:
id = 26
house = data['houses'][id-1]

print(f"House ID: {house['id']}")
print(f"Bedrooms: {house['bedrooms']}")
print(f"Bathrooms: {house['bathrooms']}")
print(f"Price: {house['price']}")
print(f"Square Meters: {house['square_meters']}")
print(f"Location: {house['location']}")

House ID: 26
Bedrooms: 1
Bathrooms: 1
Price: 100k
Square Meters: 50
Location: Santa Coloma de Gramenet


---

<h1><a name="section-two"> 2. Sistema de diàleg senzill </a></h1>


En aquesta primera part crearem un sistema senzill que mostrarà les preguntes predefinides i, a continuació l'usuari haurà d'introduir la resposta. Tractarem de manera diferent les preguntes en què la resposta siga un número i les preguntes d'opció múltiple.

Intenteu entendre el funcionament, ja que treballarem sobre aquest codi.

In [ ]:
def print_question(prompt, possible_options = []):
    print(prompt)
    if not len(possible_options) == 0:
      print("Options:", ", ".join(possible_options))

def initialize_available_options(house_data, available_options):
    for house in house_data['houses']:
        for key, value in house.items():
            available_options.setdefault(key, set()).add(value)

def preprocess_answer(answer):
    answer = nltk.word_tokenize(answer)
    return answer

def get_numerical_value(tok_answer):
    for token in tok_answer:
        if token.isnumeric() or token[:-1].isnumeric():
            return token
    return ''

#Exemple de frase tokenitzada
print(preprocess_answer('Vull un pis de 35k €'))

['Vull', 'un', 'pis', 'de', '35k', '€']


In [ ]:
def process_numerical_question(question):
    #PROBLEMA: QUALSEVOL RESPOSTA NuMèRICA POT SER ACCEPTADA
    print_question(question['question'])
    while True:
      answer = input(question['prompt'])
      tok_answer = preprocess_answer(answer)
      value = get_numerical_value(tok_answer)
      if not value == '':
        return value

def process_multichoice_question(question, options):
    print_question(question['question'], options)
    while True:
      answer = input(question['prompt'])
      if answer in options:
        return answer

user_preferences, available_options = {}, {}
initialize_available_options(data, available_options)

print(data['start_message'])
for question in data['questions']:
  answer_key = question['answer_key']
  possible_options = list(available_options.get(answer_key))

  if question['type'] == 'numerical':
    answer = process_numerical_question(question)
  else:
    answer = process_multichoice_question(question, possible_options)

  user_preferences[answer_key] = answer

Welcome to the House Buying Assistant!
How many bedrooms do you need?
Enter your choice (1 - 5 bedrooms): 1
How many bathrooms do you need?
Enter your choice (1 - 3 bathrooms): 1
What is your budget for the house?
Enter your choice (1k - 999k euros):1
How many square meters do you need?
Enter your choice (30 - 200 meters)1
Which city or neighborhood would you prefer?
Options: Barcelona, Santa Coloma de Gramenet, L'Hospitalet de Llobregat, Esplugues de Llobregat
Enter your choiceBarcelona


In [ ]:
user_preferences

{'bedrooms': '1',
 'bathrooms': '1',
 'price': '1',
 'square_meters': '1',
 'location': 'Barcelona'}

Tal com es veu, a la pregunta de "price" i de "square_meters" hem pogut respondre 1, mentre que les respostes haurien d'estar controlades dins d'un rang. Esperem poder modificar la implementació per resoldre aquest problema posteriorment.



---


 <h1><a name="ex-two"><center> ✏ Exercici 2 ✏ </a></h1>

A continuació millorarem un poc el sistema. Us proposem que afegiu la frase de benvinguda que tenim guardada al fitxer JSON. És a dir, abans de mostrar la primera pregunta el sistema ens donarà la benvinguda.

In [ ]:
#TO DO
class Chatbot():
      def __init__(self, data):
          self.data = data
          self.available_options = {}
          initialize_available_options(data, self.available_options)

      def init(self):
          self.user_preferences = {}
          # Missatge de benvinguda
          print(self.data['start_message'])

          for question in self.data['questions']:
            answer_key = question['answer_key']
            possible_options = list(available_options.get(answer_key))

            if question['type'] == 'numerical':
              answer = self.process_numerical_question(question)
            else:
              answer = self.process_multichoice_question(question, possible_options)

            self.user_preferences[answer_key] = answer

      def process_numerical_question(self, question):
          #PROBLEMA: NOMÉS ACCEPTA RESPOSTES EXISTENTS
          print_question(question['question'])
          while True:
            answer = input(question['prompt'])
            tok_answer = preprocess_answer(answer)
            value = get_numerical_value(tok_answer)
            if not value == '' and value in available_options[question["answer_key"]]:
              return value

      def process_multichoice_question(self, question, options):
          print_question(question['question'], options)
          while True:
            answer = input(question['prompt'])
            if answer in options:
              return answer

In [ ]:
user_1 = Chatbot(data)
user_1.init()
user_preferences = user_1.user_preferences

Welcome to the House Buying Assistant!
How many bedrooms do you need?
Enter your choice (1 - 5 bedrooms): 1
How many bathrooms do you need?
Enter your choice (1 - 3 bathrooms): 1
What is your budget for the house?
Enter your choice (1k - 999k euros):1
Enter your choice (1k - 999k euros):1k
Enter your choice (1k - 999k euros):120k
How many square meters do you need?
Enter your choice (30 - 200 meters)80
Which city or neighborhood would you prefer?
Options: Barcelona, Santa Coloma de Gramenet, L'Hospitalet de Llobregat, Esplugues de Llobregat
Enter your choiceBarcelona


In [ ]:
user_preferences

{'bedrooms': '1',
 'bathrooms': '1',
 'price': '120k',
 'square_meters': '80',
 'location': 'Barcelona'}

Com a objectiu de l'exercici, hem pogut imprimir la frase de benvinguda en inicialitzar la conversació. Per altra banda, hem intentat corregir l'error que hi havia l'exercici 1, ara la resposta de les preguntes numèriques està controlada, però massa controlada per tal de només poder acceptar respostes que són disponibles a les opcions, en lloc de ser un número qualsevol dins del rang. Intentarem també corregir-lo més tard.

---


 <h1><a name="ex-three"><center> ✏ Exercici 3 ✏ </a></h1>

En aquest exercici us demanem que permeteu a l'usuari abandonar el programa quan ho desitge. Per exemple, si l'usuari escriu `quit`, que el sistema s'acomiade amb l'oració de comiat que trobareu al JSON i finalitze el programa.


🙃 `sys.exit()` genera una excepció SystemExit.

In [ ]:
#TO DO
class Chatbot():
      def __init__(self, data):
          self.data = data
          self.available_options = {}
          initialize_available_options(data, self.available_options)

      def init(self):
          self.user_preferences = {}
          print(self.data['start_message'])

          for question in self.data['questions']:
            answer_key = question['answer_key']
            possible_options = list(self.available_options.get(answer_key))

            if question['type'] == 'numerical':
              answer = self.process_numerical_question(question)
            else:
              answer = self.process_multichoice_question(question, possible_options)

            self.user_preferences[answer_key] = answer

          print(self.data['end_message'])

      def process_numerical_question(self, question):
          print_question(question['question'])
          while True:
            answer = input(question['prompt'])
            # Poder finalitzar el programa
            if answer.lower() == 'quit':
                print(self.data['end_message'])
                sys.exit()
            else:
                tok_answer = preprocess_answer(answer)
                value = get_numerical_value(tok_answer)
                if not value == '' and value in self.available_options[question["answer_key"]]:
                  return value

      def process_multichoice_question(self, question, options):
          print_question(question['question'], options)
          while True:
            answer = input(question['prompt'])
            # Poder finalitzar el programa
            if answer.lower()  == 'quit':
                print(self.data['end_message'])
                sys.exit()
            elif answer in options:
                return answer

In [ ]:
user_2 = Chatbot(data)
user_2.init()

Welcome to the House Buying Assistant!
How many bedrooms do you need?
Enter your choice (1 - 5 bedrooms): 1
How many bathrooms do you need?
Enter your choice (1 - 3 bathrooms): quit
Thank you for using the House Buying Assistant. Goodbye!


SystemExit: 

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Funciona també si escriviu `Quit`?

Sí, amb answer.lower() es converteix l'entrada de l'usuari en minúscules.

---

 <h1><a name="ex-four"><center> ✏ Exercici 4 ✏ </a></h1>

Fins ara tenim a `user_preferences` les opcions que l'usuari ha escollit.







In [ ]:
user_preferences

{'bedrooms': '1',
 'bathrooms': '1',
 'price': '120k',
 'square_meters': '80',
 'location': 'Barcelona'}

Per tant, ara haurem de revisar si l'agència disposa d'alguna casa disponible amb aquestes característiques. Per això us proposem completar la funció `find_suitable_houses`. Aquesta funció tornarà una llista amb tants diccionaris com cases disponibles. Per exemple si segons les preferències de l'usuari només està disponible la casa amb id 3, `suitable_house` serà:


```
[{'id': 3, 'bedrooms': '2', 'bathrooms': '1', 'price': '160k', 'square_meters': '80', 'location': 'Santa Coloma de Gramenet'}]
```

In [ ]:
def convert_into_num(num):
    if num[-1].lower() == 'k':
        num = float(num[:-1]) * 1000
    return int(num)

filters= {
    'bedrooms': lambda answer, house: int(house) >= int(answer),
    'bathrooms': lambda answer, house: int(house) >= int(answer),
    'price':lambda answer, house: convert_into_num(house) <= convert_into_num(answer),
    'square_meters': lambda answer, house: int(house) >= int(answer),
    'location': lambda answer, house: house == answer
    }

In [ ]:
#TODO
def find_suitable_houses(data, user_preferences):
    suitable_houses = []
    for house in data['houses']:
      if all(filters[key](user_preferences[key], house[key]) for key in user_preferences if key in filters):
        filtered_house_info = house
        suitable_houses.append(filtered_house_info)
    return suitable_houses

In [ ]:
find_suitable_houses(data, user_preferences)

[{'id': 22,
  'type': 'rent',
  'bedrooms': '4',
  'bathrooms': '2',
  'price': '3000',
  'square_meters': '100',
  'floor': '0',
  'elevator': 'No',
  'commercial_use': 'Yes',
  'terrace': 'No',
  'location': 'Barcelona'}]

Hem comprovat manualment, amb unes quantes proves, que el filtratge és correcte.

---



Una vegada tingueu la funció `find_suitable_houses` programada, podreu executar el següent *script* que us mostrarà, si n'hi ha, la casa o cases disponibles amb les característiques que heu triat.

In [ ]:
def print_suitable_houses(suitable_houses):
  if suitable_houses:
    print("\nBased on your preferences, the most suitable houses are:")
    for house in suitable_houses:
      print(f"House ID: {house['id']}")
      print(f"Type: {house['type']}")
      print("Bedrooms:", house['bedrooms'])
      print("Bathrooms:", house['bathrooms'])
      print("Price:", house['price'], "euros")
      print("Square Meters:", house['square_meters'], "m^2")
      print("Location:", house['location'])
      print()
  else:
    print("\nSorry, no suitable houses match your preferences. \n")


suitable_houses = find_suitable_houses(data, user_preferences)
print_suitable_houses(suitable_houses)


Based on your preferences, the most suitable houses are:
House ID: 22
Type: rent
Bedrooms: 4
Bathrooms: 2
Price: 3000 euros
Square Meters: 100 m^2
Location: Barcelona



<h1><a name="section-three"> 3. Millorem el sistema de diàleg </a></h1>


---


 <h1><a name="ex-five"><center> ✏ Exercici 5 ✏ </a></h1>

Tal com haureu comprovat sempre esteu obligats a triar una opció. En aquest últim exercici proposem que afegiu l'opció de no triar-ne cap. És a dir, si ens és igual el nombre d'habitacions, podrem posar per exemple `any` i el sistema ens farà la següent pregunta.

In [ ]:
#TO DO
class Chatbot():
      def __init__(self, data):
          self.data = data
          self.available_options = {}
          initialize_available_options(data, self.available_options)
          self.filters = {
              'bedrooms': lambda answer, house: int(house) >= int(answer),
              'bathrooms': lambda answer, house: int(house) >= int(answer),
              'price':lambda answer, house: convert_into_num(house) <= convert_into_num(answer),
              'square_meters': lambda answer, house: int(house) >= int(answer),
              'location': lambda answer, house: house == answer
              }
          self.any = []

      def init(self):
          self.user_preferences = {}
          print(self.data['start_message'])

          for question in self.data['questions']:
            answer_key = question['answer_key']
            possible_options = list(self.available_options.get(answer_key))

            if question['type'] == 'numerical':
              answer = self.process_numerical_question(question)
            else:
              answer = self.process_multichoice_question(question, possible_options)

            self.user_preferences[answer_key] = answer

          self.find_suitable_houses()
          print_suitable_houses(self.suitable_houses)

          print(self.data['end_message'])

      def process_numerical_question(self, question):
          print_question(question['question'])
          while True:
            answer = input(question['prompt'])
            # Poder finalitzar el programa
            if answer.lower() == 'quit':
                print(self.data['end_message'])
                sys.exit()
            # Opció de respondre any
            elif answer.lower() == 'any':
                self.any.append(question['answer_key'])
                return None
            else:
                tok_answer = preprocess_answer(answer)
                value = get_numerical_value(tok_answer)
                if not value == '' and value in self.available_options[question["answer_key"]]:
                  return value

      def process_multichoice_question(self, question, options):
          print_question(question['question'], options)
          while True:
            answer = input(question['prompt'])
            # Poder finalitzar el programa
            if answer.lower()  == 'quit':
                print(self.data['end_message'])
                sys.exit()
            # Opció de respondre any
            elif answer.lower() == 'any':
                self.any.append(question['answer_key'])
                return None
            elif answer in options:
                return answer

      def find_suitable_houses(self):
          self.suitable_houses = []
          for house in self.data['houses']:
            # Filtrar les keys que estiguin en la llista de any o que compleixin el filtre
            if all((key in self.any or self.filters[key](self.user_preferences[key], house[key])) for key in self.user_preferences if key in self.filters):
              filtered_house_info = house
              self.suitable_houses.append(filtered_house_info)

In [ ]:
user_3 = Chatbot(data)
user_3.init()

Welcome to the House Buying Assistant!
How many bedrooms do you need?
Enter your choice (1 - 5 bedrooms): any
How many bathrooms do you need?
Enter your choice (1 - 3 bathrooms): any
What is your budget for the house?
Enter your choice (1k - 999k euros):Any
How many square meters do you need?
Enter your choice (30 - 200 meters)anY
Which city or neighborhood would you prefer?
Options: Barcelona, Santa Coloma de Gramenet, L'Hospitalet de Llobregat, Esplugues de Llobregat
Enter your choiceanY

Based on your preferences, the most suitable houses are:
House ID: 1
Type: rent
Bedrooms: 3
Bathrooms: 2
Price: 1200 euros
Square Meters: 100 m^2
Location: Santa Coloma de Gramenet

House ID: 2
Type: sale
Bedrooms: 4
Bathrooms: 2
Price: 250k euros
Square Meters: 120 m^2
Location: Santa Coloma de Gramenet

House ID: 3
Type: rent
Bedrooms: 2
Bathrooms: 1
Price: 800 euros
Square Meters: 80 m^2
Location: Santa Coloma de Gramenet

House ID: 4
Type: sale
Bedrooms: 1
Bathrooms: 1
Price: 100k euros
Square M

---



 <h1><a name="ex-six"><center> ✏ Exercici 6 ✏ </a></h1>

Amplieu el vostre xatbot perquè tinga en compte la següent informació sobre:

1. **Tipus d'habitatge:** Pregunteu a l'usuari si vol una casa per a compra (tipus *"sale"* a les dades) o per llogar (tipus *"rent"*). Al final de l'execució, el sistema haurá de mostrar habitatges amb el tipus seleccionat.

2. **Ingressos:** S'aconsella no dedicar més del 35% dels ingressos a l'habitatge. En cas d'escollir lloguer, preguntar a l'usuari els ingressos mensuals de la seua llar, i només mostrar aquells que estiguen per sota del 35%.

3. **Planta:** Preguntar als usuaris a quina planta volen viure. Al final de l'execució, només s'oferiràn habitatges que estiguen en aquella planta o superior.

4. **Terrassa:** Preguntar a l'usuari si vol un habitatge amb terrassa o no. En cas de voler terrassa, només s'oferiran habitatges amb terrassa. En cas contrari, oferir habitatges amb terrassa o sense.

5. **Ascensor:** Preguntar a l'usuari si vol un habitatge amb ascensor o no. En cas de voler ascensor, només s'oferiran habitatges amb ascensor. En cas contrari, oferir habitatges amb ascensor o sense.

6. **Us comercial:** Preguntar a l'usuari si vol fer servir l'habitatge com a negoci. En cas afirmatiu, només oferir habitatges amb la propietat *"commercial use"*

---





In [ ]:
def print_suitable_houses(suitable_houses):
    if suitable_houses:
        print("\nBased on your preferences, the most suitable houses are:")
        for house in suitable_houses:
            print(f"House ID: {house['id']}")
            print(f"Type: {house['type']}")
            print("Bedrooms:", house['bedrooms'])
            print("Bathrooms:", house['bathrooms'])
            print("Price:", house['price'], "euros")
            print("Square Meters:", house['square_meters'], "m^2")
            print("Floor:", house['floor'], "º")
            print("Elevator:", house['elevator'])
            print("Commercial Use:", house['commercial_use'])
            print("Terrace:", house['terrace'])
            print("Location:", house['location'])
            print()
    else:
        print("\nSorry, no suitable houses match your preferences. \n")

In [ ]:
#TO DO
class Chatbot():
      def __init__(self, data):
          self.data = data
          self.available_options = {}
          initialize_available_options(data, self.available_options)
          self.filters = {
              'type': lambda answer, house: house == answer,
              'bedrooms': lambda answer, house: int(house) >= int(answer),
              'bathrooms': lambda answer, house: int(house) >= int(answer),
              'price':lambda answer, house: convert_into_num(house) <= convert_into_num(answer) and \
                                              (convert_into_num(house) <= convert_into_num(self.user_preferences['income'])*0.35 \
                                                if self.house_type == 'rent' else True),
              'square_meters': lambda answer, house: int(house) >= int(answer),
              'floor': lambda answer, house: int(house) >= int(answer),
              'elevator': lambda answer, house: house == answer or answer == "No",
              'commercial_use': lambda answer, house: house == answer or answer == "No",
              'terrace': lambda answer, house: house == answer or answer == "No",
              'location': lambda answer, house: house == answer
              }
          self.any = []

      def init(self):
          self.user_preferences = {}
          print(self.data['start_message'])

          for question in self.data['questions']:
            answer_key = question['answer_key']
            if (answer_key != 'income'):
              possible_options = list(self.available_options.get(answer_key))

            if question['type'] == 'numerical':
              if (answer_key != 'income') or \
               (answer_key == 'income' and self.user_preferences['type'] != 'sale' ):
                answer = self.process_numerical_question(question)
            else:
              answer = self.process_multichoice_question(question, possible_options)

            self.user_preferences[answer_key] = answer

          self.find_suitable_houses()
          print_suitable_houses(self.suitable_houses)

          print(self.data['end_message'])

      def process_numerical_question(self, question):
          print_question(question['question'])
          while True:
            answer = input(question['prompt'])
            # Poder finalitzar el programa
            if answer.lower() == 'quit':
                print(self.data['end_message'])
                sys.exit()
            # Opció de respondre any
            elif answer.lower() == 'any':
                self.any.append(question['answer_key'])
                return None
            else:
                tok_answer = preprocess_answer(answer)
                value = get_numerical_value(tok_answer)
                #Canviem de nou aquesta condició perquè sinó pot donar errors per la clau de 'income'
                if not value == '':
                  return value

      def process_multichoice_question(self, question, options):
          print_question(question['question'], options)
          while True:
            answer = input(question['prompt'])
            # Poder finalitzar el programa
            if answer.lower()  == 'quit':
                print(self.data['end_message'])
                sys.exit()
            # Opció de respondre any
            elif answer.lower() == 'any':
                self.any.append(question['answer_key'])
                return None
            elif answer in options:
                return answer

      def find_suitable_houses(self):
          self.suitable_houses = []
          for house in self.data['houses']:
            self.house_type = house['type']
            # Filtrar les keys que estiguin en la llista de any o que compleixin el filtre
            if all((key in self.any or self.filters[key](self.user_preferences[key], house[key])) for key in self.user_preferences if key in self.filters):
              filtered_house_info = house
              self.suitable_houses.append(filtered_house_info)

In [ ]:
with open('/content/drive/MyDrive/3r_1/TVD/P1/dades/house_data_new.json') as f:
  new_data = json.load(f)
user_3 = Chatbot(new_data)
user_3.init()

Welcome to the House Buying Assistant!
Do you want a house to buy or to rent?
Options: rent, sale
Enter your choice: rent
What is your household's total monthly income?
Enter your choice (1k - 999k euros): 3k
How many bedrooms do you need?
Enter your choice (1 - 5 bedrooms): 1
How many bathrooms do you need?
Enter your choice (1 - 3 bathrooms): 2
What is your budget for the house?
Enter your choice (1k - 999k euros): 1k
How many square meters do you need?
Enter your choice (30 - 200 meters): 30
On which floor would you like to live?
Enter your choice (0 - 8): 0
Is it important for the property to have an elevator?
Options: Yes, No
Enter your choice: No
Do you want to use the property for business purposes?
Options: No, Yes
Enter your choice: No
Would you like the property to have a terrace?
Options: No, Yes
Enter your choice: Yes
Which city or neighborhood would you prefer?
Options: Barcelona, Santa Coloma de Gramenet, L'Hospitalet de Llobregat, Esplugues de Llobregat
Enter your choice

In [ ]:
user_3.user_preferences

{'type': 'rent',
 'income': '3k',
 'bedrooms': '1',
 'bathrooms': '2',
 'price': '1k',
 'square_meters': '30',
 'floor': '0',
 'elevator': 'No',
 'commercial_use': 'No',
 'terrace': 'Yes',
 'location': 'Barcelona'}

 <h1><a name="ex-seven"><center> ✏ Exercici 7 ✏ </a></h1>

Ara ja teniu un sistema funcional. Aplicant el mètode vist a classe (Exemples, Camins, Prototipat i Proves) milloreu aquest sistema per donar una millor experiència als usuaris.

In [ ]:
user_4 = Chatbot(new_data)
user_4.init()

Welcome to the House Buying Assistant!
Do you want a house to buy or to rent?
Options: rent, sale
Enter your choice: I want a house to rent
Enter your choice: rent
What is your household's total monthly income?
Enter your choice (1k - 999k euros): 3k aprox.
How many bedrooms do you need?
Enter your choice (1 - 5 bedrooms): quit
Thank you for using the House Buying Assistant. Goodbye!


SystemExit: 

In [ ]:
#Problema 1: L'usuari pot donar una resposta llarga de manera que el sistema la interpreti malament.
#Solució 1: Introduir models de reconeixement de llenguatge natural més intel·ligents.
#Solució 2: Avisar a l'usuari que la resposta sigui curta i clara.

def init(self):
    self.user_preferences = {}
    print(self.data['start_message'])
    print('For each question, I am going to offer you the available options.')
    print('Ensure that your answer is short and clear, please!')
    print('You can always answer QUIT to quit the conversation or ANY to include all possibilities.')

    for question in self.data['questions']:
        answer_key = question['answer_key']
        if (answer_key != 'income'):
            possible_options = list(self.available_options.get(answer_key))

        if question['type'] == 'numerical':
            if (answer_key != 'income') or \
            (answer_key == 'income' and self.user_preferences['type'] != 'sale' ):
                answer = self.process_numerical_question(question)
        else:
            answer = self.process_multichoice_question(question, possible_options)

        self.user_preferences[answer_key] = answer

    self.find_suitable_houses()
    print_suitable_houses(self.suitable_houses)

    print(self.data['end_message'])

Chatbot.init = init

In [ ]:
user_4 = Chatbot(new_data)
user_4.init()

Welcome to the House Buying Assistant!
For each question, I am going to offer you the available options.
Ensure that your answer is short and clear, please!
You can always answer QUIT to quit the conversation or ANY to include all possibilities.
Do you want a house to buy or to rent?
Options: rent, sale
Enter your choice: quit
Thank you for using the House Buying Assistant. Goodbye!


SystemExit: 

Veiem que en inicialitzar la conversació es fa un avís a l'usuari per dir-li que ha de respondre de forma clara i curta.

In [ ]:
user_4 = Chatbot(new_data)
user_4.init()

Welcome to the House Buying Assistant!
For each question, I am going to offer you the available options.
Ensure that your answer is short and clear, please!
You can always answer QUIT to quit the conversation or ANY to include all possibilities.
Do you want a house to buy or to rent?
Options: rent, sale
Enter your choice: rent
What is your household's total monthly income?
Enter your choice (1k - 999k euros): 1
How many bedrooms do you need?
Enter your choice (1 - 5 bedrooms): quit
Thank you for using the House Buying Assistant. Goodbye!


SystemExit: 

In [ ]:
#Problema 2: Per a les preguntes numèriques, la resposta no està controlada dins d'un rang.
#Solució: Afegir condicions de verificació.
def extract_range(oracion):
    #Precondició: l'oració ha de contenir 2 números que representen un rang.
    tokens = preprocess_answer(oracion)
    numeros = [int(convert_into_num(token)) for token in tokens if token.isnumeric() or token[:-1].isnumeric()]
    return range(numeros[0], numeros[1]+1)

def process_numerical_question(self, question):
    print_question(question['question'])
    while True:
        answer = input(question['prompt'])
        range = extract_range(question['prompt'])
        # Poder finalitzar el programa
        if answer.lower() == 'quit':
            print(self.data['end_message'])
            sys.exit()
        # Opció de respondre any
        elif answer.lower() == 'any':
            self.any.append(question['answer_key'])
            return None
        else:
            tok_answer = preprocess_answer(answer)
            value = get_numerical_value(tok_answer)
            if value != '' and convert_into_num(value) in range:
                return value

Chatbot.process_numerical_question = process_numerical_question

In [ ]:
user_4 = Chatbot(new_data)
user_4.init()

Welcome to the House Buying Assistant!
For each question, I am going to offer you the available options.
Ensure that your answer is short and clear, please!
You can always answer QUIT to quit the conversation or ANY to include all possibilities.
Do you want a house to buy or to rent?
Options: rent, sale
Enter your choice: rent
What is your household's total monthly income?
Enter your choice (1k - 999k euros): 1
Enter your choice (1k - 999k euros): 100
Enter your choice (1k - 999k euros): 999
Enter your choice (1k - 999k euros): 1k
How many bedrooms do you need?
Enter your choice (1 - 5 bedrooms): 0
Enter your choice (1 - 5 bedrooms): 6
Enter your choice (1 - 5 bedrooms): 3
How many bathrooms do you need?
Enter your choice (1 - 3 bathrooms): quit
Thank you for using the House Buying Assistant. Goodbye!


SystemExit: 

In [ ]:
user_4 = Chatbot(new_data)
user_4.init()

Welcome to the House Buying Assistant!
For each question, I am going to offer you the available options.
Ensure that your answer is short and clear, please!
You can always answer QUIT to quit the conversation or ANY to include all possibilities.
Do you want a house to buy or to rent?
Options: rent, sale
Enter your choice: any
What is your household's total monthly income?
Enter your choice (1k - 999k euros): any
How many bedrooms do you need?
Enter your choice (1 - 5 bedrooms): any
How many bathrooms do you need?
Enter your choice (1 - 3 bathrooms): any
What is your budget for the house?
Enter your choice (1k - 999k euros): any
How many square meters do you need?
Enter your choice (30 - 200 meters): any
On which floor would you like to live?
Enter your choice (0 - 8): any
Is it important for the property to have an elevator?
Options: Yes, No
Enter your choice: any
Do you want to use the property for business purposes?
Options: No, Yes
Enter your choice: any
Would you like the property 

SystemExit: 

In [62]:
#Problema 3: Per a les preguntes categòriques amb més de 2 opcions, l'usuari només pot seleccionar o bé una o bé totes les opcions.
#Solució: Crear noves regles que permeti donar més opcions.
def extract_answers(oracion, options):
    #Precondició: S'espera que l'oració contingui les opcions, sinó retorna un conjunt buit
    answer= set()
    oracion_set = preprocess_answer(oracion)

    for option in options:
      option_set = set(option.split())

      if option_set.issubset(oracion_set):
        answer = answer.union(set([option]))
    return answer

def process_multichoice_question(self, question, options):
    print_question(question['question'], options)
    while True:
        answer = input(question['prompt'])
        # Poder finalitzar el programa
        if answer.lower()  == 'quit':
            print(self.data['end_message'])
            sys.exit()
        # Opció de respondre any
        elif answer.lower() == 'any':
            self.any.append(question['answer_key'])
            return None
        elif len(extract_answers(answer, options)) > 0:
            return extract_answers(answer, options)

def __init__(self, data):
    self.data = data
    self.available_options = {}
    initialize_available_options(data, self.available_options)
    self.filters = {
        'type': lambda answer, house: house in answer,
        'bedrooms': lambda answer, house: int(house) >= int(answer),
        'bathrooms': lambda answer, house: int(house) >= int(answer),
        'price':lambda answer, house: convert_into_num(house) <= convert_into_num(answer) and \
                                        (convert_into_num(house) <= convert_into_num(self.user_preferences['income'])*0.35 \
                                        if self.house_type == 'rent' else True),
        'square_meters': lambda answer, house: int(house) >= int(answer),
        'floor': lambda answer, house: int(house) >= int(answer),
        'elevator': lambda answer, house: house in answer or "No" in answer,
        'commercial_use': lambda answer, house: house in answer or "No" in answer,
        'terrace': lambda answer, house: house in answer or "No" in answer,
        'location': lambda answer, house: house in answer
        }
    self.any = []

Chatbot.__init__ = __init__
Chatbot.process_multichoice_question = process_multichoice_question

In [54]:
user_4 = Chatbot(new_data)
user_4.init()

Welcome to the House Buying Assistant!
For each question, I am going to offer you the available options.
Ensure that your answer is short and clear, please!
You can always answer QUIT to quit the conversation or ANY to include all possibilities.
Do you want a house to buy or to rent?
Options: rent, sale
Enter your choice: I want a house to rent.
What is your household's total monthly income?
Enter your choice (1k - 999k euros): 3k aprox.
How many bedrooms do you need?
Enter your choice (1 - 5 bedrooms): 1
How many bathrooms do you need?
Enter your choice (1 - 3 bathrooms): 1
What is your budget for the house?
Enter your choice (1k - 999k euros): 2k
How many square meters do you need?
Enter your choice (30 - 200 meters): any
On which floor would you like to live?
Enter your choice (0 - 8): any
Is it important for the property to have an elevator?
Options: Yes, No
Enter your choice: any
Do you want to use the property for business purposes?
Options: No, Yes
Enter your choice: No
Would yo

In [55]:
user_4.user_preferences

{'type': {'rent'},
 'income': '3k',
 'bedrooms': '1',
 'bathrooms': '1',
 'price': '2k',
 'square_meters': None,
 'floor': None,
 'elevator': None,
 'commercial_use': {'No'},
 'terrace': {'Yes'},
 'location': {'Barcelona', 'Santa Coloma de Gramenet'}}

Tal com es veu, tant Barcelona com Santa Coloma de Gramenet estan inclosos en la preferència de l'usuari. A més, el sistema pot extreure aquestes ciutats claus dins d'una oració ben formulada.

In [56]:
user_4 = Chatbot(new_data)
user_4.init()

Welcome to the House Buying Assistant!
For each question, I am going to offer you the available options.
Ensure that your answer is short and clear, please!
You can always answer QUIT to quit the conversation or ANY to include all possibilities.
Do you want a house to buy or to rent?
Options: rent, sale
Enter your choice: hi
Enter your choice: Rent
Enter your choice: quit
Thank you for using the House Buying Assistant. Goodbye!


SystemExit: 

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [63]:
#Problema 4: Quan l'usuari introduiex malament la resposta, no sap quin error ha comès.
#Solució: Afegir condicions que identifica alguns possibles problemes i donar un avís.

def process_multichoice_question(self, question, options):
    print_question(question['question'], options)
    while True:
        answer = input(question['prompt'])
        # Poder finalitzar el programa
        if answer.lower()  == 'quit':
            print(self.data['end_message'])
            sys.exit()
        # Opció de respondre any
        elif answer.lower() == 'any':
            self.any.append(question['answer_key'])
            return None
        elif len(extract_answers(answer, options)) > 0:
            return extract_answers(answer, options)
        else:
            print('You are not giving any of the available options, please ensure that your answer does not have any mistake.')

def process_numerical_question(self, question):
    print_question(question['question'])
    while True:
        answer = input(question['prompt'])
        range = extract_range(question['prompt'])
        # Poder finalitzar el programa
        if answer.lower() == 'quit':
            print(self.data['end_message'])
            sys.exit()
        # Opció de respondre any
        elif answer.lower() == 'any':
            self.any.append(question['answer_key'])
            return None
        else:
            tok_answer = preprocess_answer(answer)
            value = get_numerical_value(tok_answer)
            if value == '':
                print('You are not introducing any numerical number, please ensure that your answer does not have any mistake.')
            elif convert_into_num(value) in range:
                return value
            elif convert_into_num(value) not in range:
                print('Your answer is out of range, please ensure that your answer does not have any mistake.')

Chatbot.process_numerical_question = process_numerical_question
Chatbot.process_multichoice_question = process_multichoice_question

In [65]:
user_4 = Chatbot(new_data)
user_4.init()

Welcome to the House Buying Assistant!
For each question, I am going to offer you the available options.
Ensure that your answer is short and clear, please!
You can always answer QUIT to quit the conversation or ANY to include all possibilities.
Do you want a house to buy or to rent?
Options: rent, sale
Enter your choice: hi
You are not giving any of the available options, please ensure that your answer does not have any mistake.
Enter your choice: Rent
You are not giving any of the available options, please ensure that your answer does not have any mistake.
Enter your choice: rent
What is your household's total monthly income?
Enter your choice (1k - 999k euros): 1
Your answer is out of range, please ensure that your answer does not have any mistake.
Enter your choice (1k - 999k euros): 999
Your answer is out of range, please ensure that your answer does not have any mistake.
Enter your choice (1k - 999k euros): hi
You are not introducing any numerical number, please ensure that your 

SystemExit: 

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Veiem que cada vegada que es repeteix la pregunta (perquè hi ha algun error), surt un mitssage d'avís, juntament amb una justificació de l'error.

In [69]:
auser_4 = Chatbot(new_data)
user_4.init()

Welcome to the House Buying Assistant!
For each question, I am going to offer you the available options.
Ensure that your answer is short and clear, please!
You can always answer QUIT to quit the conversation or ANY to include all possibilities.
Do you want a house to buy or to rent?
Options: rent, sale
Enter your choice: rent
What is your household's total monthly income?
Enter your choice (1k - 999k euros): 2k
How many bedrooms do you need?
Enter your choice (1 - 5 bedrooms): 1
How many bathrooms do you need?
Enter your choice (1 - 3 bathrooms): 1
What is your budget for the house?
Enter your choice (1k - 999k euros): 3k
How many square meters do you need?
Enter your choice (30 - 200 meters): any
On which floor would you like to live?
Enter your choice (0 - 8): any
Is it important for the property to have an elevator?
Options: Yes, No
Enter your choice: AnY
Do you want to use the property for business purposes?
Options: No, Yes
Enter your choice: any
Would you like the property to ha

In [68]:
user_4.user_preferences

{'type': {'rent'},
 'income': '1k',
 'bedrooms': '1',
 'bathrooms': '1',
 'price': '3k',
 'square_meters': None,
 'floor': None,
 'elevator': None,
 'commercial_use': None,
 'terrace': None,
 'location': None}

Viem que encara que el meu pressupost és 3k, el sistema només recomana cases que tenen un preu inferior al 0.35*2k. La restricció més restringida té més prioritat.

In [70]:
#Problema 5: Hi ha dos preguntes per restringir el pressupost, pot cometre confusions.
#Solució: En el cas que l'usuari doni un pressupost major al 35% dels seus ingressos mensuals...
#...Preguntar si està segur o no.
def convert_into_bool(string):
    if string.lower() == 'yes':
        return True
    elif string.lower() == 'no':
        return False

def init(self):
    self.user_preferences = {}
    print(self.data['start_message'])
    print('For each question, I am going to offer you the available options.')
    print('Ensure that your answer is short and clear, please!')
    print('You can always answer QUIT to quit the conversation or ANY to include all possibilities.')

    for question in self.data['questions']:
        answer_key = question['answer_key']
        if (answer_key != 'income'):
            possible_options = list(self.available_options.get(answer_key))

        if question['type'] == 'numerical':
            if (answer_key != 'income') or \
            (answer_key == 'income' and self.user_preferences['type'] != 'sale' ):
                answer = self.process_numerical_question(question)

            if answer_key == 'price' and self.user_preferences['type'] != 'sale' and 'income' not in self.any:
                rich = False
                while not rich and convert_into_num(answer) > convert_into_num(self.user_preferences['income'])*0.35:
                    print('We recommend that you do not spend more than 35% of your monthly income on rent.')
                    rich = convert_into_bool(input('Are you sure you can keep the same budget? (Enter Yes or No)'))
                    if rich == None:
                        rich = False
                        print('You are not giving any of the available options, please ensure that your answer does not have any mistake.')
                    elif not rich:
                        answer = self.process_numerical_question(question)
                    else:
                        self.filters['price'] = lambda answer, house: convert_into_num(house) <= convert_into_num(answer)

        else:
            answer = self.process_multichoice_question(question, possible_options)

        self.user_preferences[answer_key] = answer

    self.find_suitable_houses()
    print_suitable_houses(self.suitable_houses)

    print(self.data['end_message'])

Chatbot.init = init


In [71]:
user_4 = Chatbot(new_data)
user_4.init()

Welcome to the House Buying Assistant!
For each question, I am going to offer you the available options.
Ensure that your answer is short and clear, please!
You can always answer QUIT to quit the conversation or ANY to include all possibilities.
Do you want a house to buy or to rent?
Options: rent, sale
Enter your choice: any
What is your household's total monthly income?
Enter your choice (1k - 999k euros): 2k
How many bedrooms do you need?
Enter your choice (1 - 5 bedrooms): 1
How many bathrooms do you need?
Enter your choice (1 - 3 bathrooms): 1
What is your budget for the house?
Enter your choice (1k - 999k euros): 3k
We recommend that you do not spend more than 35% of your monthly income on rent.
Are you sure you can keep the same budget? (Enter Yes or No)No
What is your budget for the house?
Enter your choice (1k - 999k euros): 5k
We recommend that you do not spend more than 35% of your monthly income on rent.
Are you sure you can keep the same budget? (Enter Yes or No)Yes
How ma

In [72]:
user_4.user_preferences

{'type': None,
 'income': '2k',
 'bedrooms': '1',
 'bathrooms': '1',
 'price': '5k',
 'square_meters': None,
 'floor': None,
 'elevator': None,
 'commercial_use': None,
 'terrace': None,
 'location': None}

Veiem que ara quan hi existeix una ambigüitat, el sistema pregunta a l'usuari per assegurar-se si vol que la resposta del pressupost otorgui la prioritat. Durant la conservació anterior, hem respòs que si i veiem que el sistema ha pogut ignorar la restricció dels ingressos mensuals.

<h1><a name="section-four"> 4. Lliurable </a></h1>

Heu d'entregar un document PDF de com a **màxim 10 pàgines** que incloga els resultats de tots els exercicis així com una explicació de cadascun dels resultats i de la modificació que heu fet. L'estructura del document és:

1. Introducció.
2. Resultats (amb raonament).
3. Conversation flow del vostre sistema final.
4. Conclusions.

No cal que afegiu el vostre codi al document, podeu entregar el *notebook* juntament amb el document.


Conversation flow: graf de preguntes i opcions resposta (transparències teoria)

---